In [1]:
import multiprocessing
import os
from netCDF4 import Dataset
import numpy as np
import requests
import subprocess as sp
from functools import partial
import glob
import pandas as pd
import datetime
import pygrib
from osgeo import gdal

def arr2raster(dst, arr, lons, lats):
    lon_diff= lons[1]- lons[0]
    lat_diff= lats[1]- lats[0]
    cols= arr.shape[1]
    rows= arr.shape[0]
    originX= lons[0]
    originY= lats[-1]
    driver= gdal.GetDriverByName('GTiff')
    outdata= driver.Create(dst, cols, rows, gdal.GDT_Float32)
    outdata.SetGeoTransform((originX, lon_diff, 0, originY, 0, -lat_diff))
    outdata.SetProjection('EPSG:4326')
    outdata.GetRasterBand(1).WriteArray(arr)
    
def prefix_gaugeCorr(dt):
    prefix= 'https://mtarchive.geol.iastate.edu/%04d/%02d/%02d/mrms/ncep/GaugeCorr_QPE_01H/GaugeCorr_QPE_01H_00.00_%s.grib2.gz'%(
                                                dt.year,
                                                dt.month,
                                                dt.day,
                                                dt.strftime('%Y%m%d-%H%M%S'))
    return prefix

def prefix_precipFlag(dt):
    prefix= 'https://mtarchive.geol.iastate.edu/%04d/%02d/%02d/mrms/ncep/PrecipFlag/PrecipFlag_00.00_%s.grib2.gz'%(
                                                dt.year,
                                                dt.month,
                                                dt.day,
                                                dt.strftime('%Y%m%d-%H%M%S'))
    return prefix

def single(dt):
        try:
            gdal.Open(dt.strftime("%H%m%d%H%M%S.tif")).ReadAsArray()
            print('Found !')
            
        except AttributeError:
#             download MRMS Gauge corrected
            prefix= prefix_precipFlag(dt)

            os.system('wget -O /media/scratch/ZhiLi/MRMS/PrecipFlag/%s %s'%(dt.strftime('%Y%m%d%H%M%S.grib2.gz'), prefix))

            os.system('gunzip /media/scratch/ZhiLi/MRMS/PrecipFlag/%s'%dt.strftime('%Y%m%d%H%M%S.grib2.gz'))

#             arr2raster(dt.strftime('/media/scratch/ZhiLi/MRMS/%Y%m%d%H%M%S.tif'), val, lon[0,:], lat[:,0])
            
            #gdal_translate -tr 0.1 0.1 -r nearest %s.grib2 %s.tif
            os.system('gdal_translate %s.grib2 %s.tif'%(dt.strftime('/media/scratch/ZhiLi/MRMS/PrecipFlag/%Y%m%d%H%M%S'),
                                                                dt.strftime('/media/scratch/ZhiLi/MRMS/PrecipFlag/%Y%m%d%H%M%S')))
        
            os.system('rm /media/scratch/ZhiLi/MRMS/PrecipFlag/%s.grib2'%dt.strftime('%Y%m%d%H%M%S'))
            print('finish %s!'%dt.strftime('%Y%m%d%H%M%S.grib2.gz'))
        


def main():
#         lists= a.split(',')
        #f= open('anomalies.txt','r')
#         lists= f.readlines(0)
#         f.close()
        lists= pd.date_range('20160130', '20200101', freq='H')
        with multiprocessing.Pool(30) as pool:
            fnames= pool.map(single, lists)
        return fnames

parttern : https://mtarchive.geol.iastate.edu/2015/06/01/mrms/ncep/GaugeCorr_QPE_01H/GaugeCorr_QPE_01H_00.00_20150601-000000.grib2.gz

In [3]:
start= time.time()
remaining= main()
end= time.time()
print('process one day costs %.2f minutes'%((end-start)/60.))

finish 20160210230000.grib2.gz!
finish 20160516150000.grib2.gz!
finish 20160609130000.grib2.gz!
finish 20160808080000.grib2.gz!
finish 20161019020000.grib2.gz!
finish 20160727090000.grib2.gz!
finish 20160528140000.grib2.gz!
finish 20160130000000.grib2.gz!


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ZhiLi/env/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-d720d5e18eb8>", line 2, in <module>
    remaining= main()
  File "<ipython-input-1-904c7e5bb599>", line 74, in main
    fnames= pool.map(single, lists)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 266, in map
    return self._map_async(func, iterable, mapstar, chunksize).get()
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 638, in get
    self.wait(timeout)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 635, in wait
    self._event.wait(timeout)
  File "/usr/lib64/python3.6/threading.py", line 551, in wait
    signaled = self._cond.wait(timeout)
  File "/usr/lib64/python3.6/threading.py", line 295, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (m

KeyboardInterrupt: 

In [7]:
# check remaining files
files= os.listdir('/media/scratch/ZhiLi/MRMS/PrecipFlag')

In [8]:
dr= pd.date_range('20160130', '20200101', freq='H').strftime('%Y%m%d%H%M%S.tif')

In [9]:
remaining= [f for i,f in enumerate(dr) if f not in list(files)]

In [10]:
len(remaining)

586

In [25]:
remaining

['20160216120000.tif',
 '20160225120000.tif',
 '20160301120000.tif',
 '20160302120000.tif',
 '20160309120000.tif',
 '20160318170000.tif',
 '20160318180000.tif',
 '20160326120000.tif',
 '20160327120000.tif',
 '20160402120000.tif',
 '20160414120000.tif',
 '20160427120000.tif',
 '20160505200000.tif',
 '20160523120000.tif',
 '20160524160000.tif',
 '20160526230000.tif',
 '20160616150000.tif',
 '20160629120000.tif',
 '20160630120000.tif',
 '20160701060000.tif',
 '20160706150000.tif',
 '20160707020000.tif',
 '20160728200000.tif',
 '20160802020000.tif',
 '20160802030000.tif',
 '20160911200000.tif',
 '20161002120000.tif',
 '20161011160000.tif',
 '20170407050000.tif',
 '20170427120000.tif',
 '20170509230000.tif',
 '20170510010000.tif',
 '20170510070000.tif',
 '20170511090000.tif',
 '20170511200000.tif',
 '20170512030000.tif',
 '20170512220000.tif',
 '20170513020000.tif',
 '20170513050000.tif',
 '20170513100000.tif',
 '20170513220000.tif',
 '20180124020000.tif',
 '20180124190000.tif',
 '201802071

In [11]:
def remedy(lists):
#         lists= a.split(',')
        #f= open('anomalies.txt','r')
#         lists= f.readlines(0)
#         f.close()
        lists=pd.to_datetime(lists, format= '%Y%m%d%H%M%S.tif')
        with multiprocessing.Pool(30) as pool:
            fnames= pool.map(single, lists)
#         return fnames

In [12]:
remedy(remaining)

finish 20180330170000.grib2.gz!
finish 20180402160000.grib2.gz!
finish 20180402090000.grib2.gz!
finish 20180401200000.grib2.gz!
finish 20180403040000.grib2.gz!
finish 20160526230000.grib2.gz!
finish 20170509230000.grib2.gz!
finish 20160414120000.grib2.gz!
finish 20180208080000.grib2.gz!
finish 20180223180000.grib2.gz!
finish 20180222070000.grib2.gz!
finish 20180224120000.grib2.gz!
finish 20180221220000.grib2.gz!
finish 20180224040000.grib2.gz!
finish 20160911200000.grib2.gz!
finish 20180330230000.grib2.gz!
finish 20160706150000.grib2.gz!
finish 20180226010000.grib2.gz!
finish 20170512030000.grib2.gz!
finish 20180226160000.grib2.gz!
finish 20160216120000.grib2.gz!
finish 20170513220000.grib2.gz!
finish 20180226060000.grib2.gz!
finish 20180220230000.grib2.gz!
finish 20180225170000.grib2.gz!
finish 20180301170000.grib2.gz!
finish 20180226110000.grib2.gz!
finish 20180208010000.grib2.gz!
finish 20180404000000.grib2.gz!
finish 20160318170000.grib2.gz!
finish 20180330190000.grib2.gz!
finish 2

finish 20190615000000.grib2.gz!
finish 20190618090000.grib2.gz!
finish 20190616120000.grib2.gz!
finish 20190616220000.grib2.gz!
finish 20190619140000.grib2.gz!
finish 20190618140000.grib2.gz!
finish 20190620000000.grib2.gz!
finish 20190618040000.grib2.gz!
finish 20190616070000.grib2.gz!
finish 20190618190000.grib2.gz!
finish 20190619100000.grib2.gz!
finish 20190615210000.grib2.gz!
finish 20190619000000.grib2.gz!
finish 20190619190000.grib2.gz!
finish 20190615160000.grib2.gz!
finish 20190616020000.grib2.gz!
finish 20190620040000.grib2.gz!
finish 20190615110000.grib2.gz!
finish 20190620090000.grib2.gz!
finish 20190616170000.grib2.gz!
finish 20190615060000.grib2.gz!
finish 20190613210000.grib2.gz!
finish 20190617180000.grib2.gz!
finish 20190617080000.grib2.gz!
finish 20190617230000.grib2.gz!
finish 20190617030000.grib2.gz!
finish 20190617130000.grib2.gz!
finish 20190620130000.grib2.gz!
finish 20190619060000.grib2.gz!
finish 20190618100000.grib2.gz!
finish 20190615010000.grib2.gz!
finish 2

finish 20190629230000.grib2.gz!
finish 20190629180000.grib2.gz!
finish 20190630090000.grib2.gz!
finish 20190630040000.grib2.gz!
finish 20190630140000.grib2.gz!
finish 20190627120000.grib2.gz!
finish 20190627220000.grib2.gz!
finish 20190627020000.grib2.gz!
finish 20190630190000.grib2.gz!
finish 20190627070000.grib2.gz!
finish 20190627170000.grib2.gz!
finish 20190629040000.grib2.gz!
finish 20190628080000.grib2.gz!
finish 20190626230000.grib2.gz!
finish 20190628180000.grib2.gz!
finish 20190628130000.grib2.gz!
finish 20190629090000.grib2.gz!
finish 20190629140000.grib2.gz!
finish 20190629190000.grib2.gz!
finish 20190630000000.grib2.gz!
finish 20190628230000.grib2.gz!
finish 20190628030000.grib2.gz!
finish 20190702010000.grib2.gz!
finish 20190630100000.grib2.gz!
finish 20190630050000.grib2.gz!
finish 20190630150000.grib2.gz!
finish 20190627130000.grib2.gz!
finish 20190627230000.grib2.gz!
finish 20190630200000.grib2.gz!
finish 20190627030000.grib2.gz!
finish 20190708180000.grib2.gz!
finish 2